In [2]:
import pandas as pd
data = pd.read_csv('train.csv')
data

,text,intent
0,Оформи профиль компании,create_company_profile
1,Сделай регистрацию фирмы онлайн,create_company_profile
2,Добавь профиль юрлица,create_company_profile
3,с нуля добавь данные новой компании,create_company_profile
4,Хочу создать аккаунт компании повторно,create_company_profile
...,...,...
2099,Сесть на мель,unknown
2100,Хлопать глазами,unknown
2101,Вешать лапшу,unknown
2102,Идти в ногу,unknown


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1. Загрузка данных
data = pd.read_csv('train.csv')
X = data['text']
y = data['intent']

# 2. Преобразование текста в числовые признаки
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_vectorized = vectorizer.fit_transform(X)

# 3. Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y, test_size=0.42, random_state=82
)

# 4. Обучение модели CatBoost
model = CatBoostClassifier(iterations=100, verbose=10)
model.fit(X_train, y_train)

# 5. Оценка модели
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Learning rate set to 0.5
0:	learn: 1.3483532	total: 183ms	remaining: 18.1s
10:	learn: 0.5725492	total: 572ms	remaining: 4.63s
20:	learn: 0.3368056	total: 951ms	remaining: 3.58s
30:	learn: 0.2101757	total: 1.32s	remaining: 2.94s
40:	learn: 0.1565642	total: 1.7s	remaining: 2.44s
50:	learn: 0.1146186	total: 2.07s	remaining: 1.99s
60:	learn: 0.0919718	total: 2.48s	remaining: 1.58s
70:	learn: 0.0757051	total: 2.91s	remaining: 1.19s
80:	learn: 0.0632698	total: 3.32s	remaining: 779ms
90:	learn: 0.0547143	total: 3.7s	remaining: 366ms
99:	learn: 0.0488897	total: 4.07s	remaining: 0us
Accuracy: 0.997737556561086
                           precision    recall  f1-score   support

                  add_ecp       1.00      1.00      1.00        93
   create_company_profile       1.00      1.00      1.00        76
          create_contract       1.00      0.99      0.99        79
create_direct_procurement       1.00      1.00      1.00        89
 create_quotation_session       1.00      1.00      1.0

In [4]:
test_phrase = ["закладка под деревом"]
test_vector = vectorizer.transform(test_phrase)
predicted_class = model.predict(test_vector)
print(f'Предсказанный класс: {predicted_class[0]}')

Предсказанный класс: ['unknown']


In [5]:
model.save_model("catboost_model.cbm")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [6]:
data["intent"].unique()

array(['create_company_profile', 'add_ecp', 'create_direct_procurement',
       'update_user_profile', 'create_quotation_session',
       'create_contract', 'manage_offer', 'unknown'], dtype=object)

In [7]:
model = CatBoostClassifier()
model.load_model("catboost_model.cbm")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Тестовый пример
sample_texts = [
    "Создать новую компанию",
    "Удалить оферту из корзины"
]

# Преобразование текста и предсказание
X_sample = vectorizer.transform(sample_texts)
preds = model.predict(X_sample)

for text, pred in zip(sample_texts, preds):
    print(f"Текст: {text} → Предсказанный интент: {pred}")

Текст: Создать новую компанию → Предсказанный интент: ['create_company_profile']
Текст: Удалить оферту из корзины → Предсказанный интент: ['manage_offer']
